In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import SimpleRNN
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Подготовка датасета

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/medium_articles.csv")
print(data)

                                            title  \
0                             Mental Note Vol. 24   
1                       Your Brain On Coronavirus   
2                                  Mind Your Nose   
3                        The 4 Purposes of Dreams   
4                Surviving a Rod Through the Head   
...                                           ...   
192363        Why do you need a cleaning service?   
192364  Daily cleaning and maintenance of bedding   
192365        Beneficial Advice on Bond Cleaning!   
192366    How I Learned Romanian in 37 Easy Steps   
192367     Trying Pimsleur Cantonese in Hong Kong   

                                                     text  \
0       Photo by Josh Riemer on Unsplash\n\nMerry Chri...   
1       Your Brain On Coronavirus\n\nA guide to the cu...   
2       Mind Your Nose\n\nHow smell training can chang...   
3       Passionate about the synergy between science a...   
4       You’ve heard of him, haven’t you? Phineas Gage... 

In [ ]:
from collections import Counter
def pars_tags (data):
  d = {}
  for el in data['tags']:
    for i in el:
      if i in d:
        d[i] += 1
      else:
        d[i] = 1
  cnt = Counter(d).most_common(10)
  return cnt

In [ ]:
def upd_data (data, tag_list):
  new_data = pd.DataFrame()
  for _, el in data.iterrows():
    for i in el['tags']:
      for j in tag_list:
        if i == j[0]:
          el['tags'] = i
          new_data = new_data.append(el, ignore_index = True)
  return new_data 

In [ ]:
tmp = data.tags.apply(lambda x: x[1:-1].split(','))
tmp_data = data.copy() 
tmp_data['tags'] = tmp
tags_list = pars_tags(tmp_data)
new_data = upd_data(tmp_data, tags_list)
print(new_data)

<ipython-input-41-5612d9c11a98>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(el, ignore_index = True)


                                                   title  \
0      How to Turn Your Popular Blog Series Into a Be...   
1              Four Exercises to Strengthen Your Writing   
2      This 10-Minute Routine Will Increase Your Clar...   
3              How to Make Your Day Job Support Your Art   
4      An Effective Five-Step Process for Writing Cap...   
...                                                  ...   
50163               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50164               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50165                Status messenger for Desktop review   
50166  The Impact of Technology on Hotel Guest Experi...   
50167  (Live — Streaming) : 2021 Soul Train Music Awards   

                                                    text  \
0      How to Turn Your Popular Blog Series Into a Be...   
1      A few months ago, I wrote an article sharing s...   
2      “Your subconscious mind works continuously, wh...   
3      Gather around kids and let me te

In [ ]:
x = new_data['text']
y = new_data['tags']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
new_data = pd.DataFrame(x)
new_data['tags'] = y
new_data = new_data[:200]
print(new_data)
train, test = train_test_split(new_data, test_size=0.33, random_state=42)

                                                  text  tags
0    How to Turn Your Popular Blog Series Into a Be...     9
1    A few months ago, I wrote an article sharing s...     9
2    “Your subconscious mind works continuously, wh...     4
3    Gather around kids and let me tell you a story...     9
4    You’ve spent hours pouring your heart and soul...     9
..                                                 ...   ...
195  Redefine Publishing\n\nStaying consistent duri...     9
196  The difference between beneficial and destruct...     7
197  Who are Medium’s Curators and What do They do?...     9
198  Section 1: List of Challenges and Related Stor...     9
199  Practical Examples\n\nIterators\n\nIn Python, ...     3

[200 rows x 2 columns]


In [ ]:
x = train["text"]
y = train["tags"]
x = np.asarray(x).astype(np.str_)
y = np.asarray(y).astype(np.float32)
test_x = test["text"]
test_y = test["tags"]
test_x = np.asarray(test_x).astype(np.str_)
test_y = np.asarray(test_y).astype(np.float32)

## Посимвольная векторизация

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(split='character')

print('Training vectorizer')
vectorizer.adapt(list(train["text"]))

Training vectorizer


In [ ]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=40000, output_dim=8, mask_zero=True))
model.add(layers.LSTM(4))
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_6 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_10 (Embedding)    (None, None, 8)           320000    
                                                                 
 lstm_11 (LSTM)              (None, 4)                 208       
                                                                 
 dense_11 (Dense)            (None, 100)               500       
                                                                 
Total params: 320,708
Trainable params: 320,708
Non-trainable params: 0
_________________________________________________________________


## Нейросетевыя архитектура LSTM (однонаправленная однослойная)

In [ ]:
model.compile(keras.optimizers.Adam(0.1), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 4, validation_data=(test_x, test_y))

Epoch 1/4
3/3 [==============================] - 52s 16s/step - loss: 4.4657 - acc: 0.1716 - val_loss: 3.5440 - val_acc: 0.6061
Epoch 2/4
3/3 [==============================] - 45s 14s/step - loss: 3.3843 - acc: 0.6418 - val_loss: 3.0210 - val_acc: 0.6061
Epoch 3/4
3/3 [==============================] - 44s 14s/step - loss: 2.9838 - acc: 0.6418 - val_loss: 2.9532 - val_acc: 0.6061
Epoch 4/4
3/3 [==============================] - 45s 14s/step - loss: 2.9351 - acc: 0.6418 - val_loss: 2.9515 - val_acc: 0.6061


## Пословная векторизация

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["text"]))

Training vectorizer


## Нейросетевыя архитектура LSTM (многослойная)

In [ ]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=40000, output_dim=64, mask_zero=True))
model.add(layers.LSTM(8, return_sequences=True))
model.add(layers.LSTM(4))
model.add(layers.Dense(100, activation = 'tanh'))
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_11 (Embedding)    (None, None, 64)          2560000   
                                                                 
 lstm_12 (LSTM)              (None, None, 8)           2336      
                                                                 
 lstm_13 (LSTM)              (None, 4)                 208       
                                                                 
 dense_12 (Dense)            (None, 100)               500       
                                                                 
 dense_13 (Dense)            (None, 100)               10100     
                                                     

In [ ]:
model.compile(keras.optimizers.Adam(0.01), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 4, validation_data=(test_x, test_y))

Epoch 1/4
3/3 [==============================] - 18s 4s/step - loss: 4.5535 - acc: 0.3134 - val_loss: 4.1383 - val_acc: 0.6061
Epoch 2/4
3/3 [==============================] - 9s 3s/step - loss: 3.9304 - acc: 0.6418 - val_loss: 3.3135 - val_acc: 0.6061
Epoch 3/4
3/3 [==============================] - 9s 3s/step - loss: 3.1915 - acc: 0.6418 - val_loss: 2.9851 - val_acc: 0.6061
Epoch 4/4
3/3 [==============================] - 8s 3s/step - loss: 2.9573 - acc: 0.6418 - val_loss: 2.9374 - val_acc: 0.6061


## Пословная векторизация

In [ ]:
vectorizer = tf.keras.layers.TextVectorization(split='whitespace')

print('Training vectorizer')
vectorizer.adapt(list(train["text"]))

Training vectorizer


## Нейросетевыя архитектура LSTM (двунаправленная)

In [ ]:
model = keras.Sequential()
model.add(vectorizer)
model.add(layers.Embedding(input_dim=16000, output_dim=32, mask_zero=True))
model.add(layers.Bidirectional(keras.layers.LSTM(8), merge_mode='concat')),
model.add(layers.Dense(100, activation = 'tanh'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_8 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_12 (Embedding)    (None, None, 32)          512000    
                                                                 
 bidirectional_6 (Bidirectio  (None, 16)               2624      
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 100)               1700      
                                                                 
Total params: 516,324
Trainable params: 516,324
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(keras.optimizers.Adam(0.1), keras.losses.SparseCategoricalCrossentropy(from_logits=True), ['acc'])
train_model = model.fit(x=x, y=y, batch_size=64, epochs = 4, validation_data=(test_x, test_y))

Epoch 1/4
3/3 [==============================] - 16s 4s/step - loss: 4.3515 - acc: 0.3134 - val_loss: 3.0214 - val_acc: 0.6061
Epoch 2/4
3/3 [==============================] - 6s 2s/step - loss: 2.9609 - acc: 0.6418 - val_loss: 2.9310 - val_acc: 0.6061
Epoch 3/4
3/3 [==============================] - 8s 3s/step - loss: 2.9212 - acc: 0.6418 - val_loss: 2.9313 - val_acc: 0.6061
Epoch 4/4
3/3 [==============================] - 6s 2s/step - loss: 2.9287 - acc: 0.6418 - val_loss: 2.9343 - val_acc: 0.6061
